In [18]:
# Install Google packages if needed (reload the kernel if necessary)
!pip install google
!pip install google-genai
!pip install google-generativeai
!pip install vertexai
!pip install pandas

In [3]:
! pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/757.5 kB ? eta -:--:--
   ---------------------------------------- 757.5/757.5 kB 8.4 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)

   ------------- -------------------------- 1/3 [distro]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [openai]
   -------------------------- ------------- 2/3 [o

In [1]:
import os
import json
import random
import time
import re
import pandas as pd
from google import genai
import openai

In [7]:
pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)

   ---- ----------------------------------- 1/9 [multidict]
   -------- ------------------------------- 2/9 [frozenlist]
   ------------- -------------------------- 3/9 [attrs]
   ------------- -------------------------- 3/9 [attrs]
   ----------------- ---------------------- 4/9 [aiohappyeyeballs]
   ---------------------- ----------------- 5/9 [yarl]
   -------------------------- ------------- 6/9 [aiosignal]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 

In [ ]:
# test la mémoire de modèle sans prompt
# Ce script utilise l'API OpenAI pour extraire des événements historiques de Paris 
import openai
import time
import json

openai.api_key = "xxxxxx" # Remplacez par votre clé API OpenAI

MODEL = "gpt-4"
BATCH_SIZE = 50
OUTPUT_FILE = "extraction_output.txt"
MESSAGES_LOG = "chat_history.json"
MAX_TOKENS_THRESHOLD = 6000

system_prompt = {
    "role": "system",
    "content": (
        "Tu es un expert en traitement des documents historiques de Paris. "
        "Tu dois extraire les événements dans ce format :\n"
        "'rue de Rivoli || Classement || Classement en 1801'\n"
        "'rue de Rivoli || Dénomination || Nouveau nom en 1802'\n"
        "N'oublie pas ce format pour chaque réponse."
    )
}

# 📓 Event list (réduit pour démo)
evenements = [
    "avenue du maine || Historique || Cette voie est indiquée sur le plan de Vaugondy (1760)",
    "impasse martini || Historique || Elle existait en 1652"
]

NB_BATCHES = len(evenements) // BATCH_SIZE + int(len(evenements) % BATCH_SIZE != 0)

# Token estimate

def estimate_tokens(msg_list):
    return sum(len(msg["content"].split()) for msg in msg_list)

messages = []
history = []

with open(OUTPUT_FILE, "w", encoding="utf-8") as f_out:
    for i in range(NB_BATCHES):
        start = i * BATCH_SIZE
        end = min(start + BATCH_SIZE, len(evenements))
        batch_text = "\n".join(evenements[start:end])
        user_msg = {"role": "user", "content": f"Voici les événements à extraire :\n{batch_text}"}

        if i == 0:
            messages.append(system_prompt)

        messages.append(user_msg)

        if estimate_tokens(messages) > MAX_TOKENS_THRESHOLD:
            print("\n⚠️ Token limite atteinte. Reboot avec prompt.")
            messages = [system_prompt, user_msg]

        print(f"\n📤 Batch {i+1} envoyé avec contexte :")
        for m in messages[-2:]:
            print(f"[{m['role'].upper()}] {m['content']}\n")

        try:
            response = openai.ChatCompletion.create(
                model=MODEL,
                messages=messages
            )
            result = response["choices"][0]["message"]["content"]

            print(f"📥 Réponse batch {i+1} :\n{result}\n")

            messages.append({"role": "assistant", "content": result})
            history.append({"user": user_msg["content"], "assistant": result})

            f_out.write(f"--- Résultats {start}-{end} ---\n")
            f_out.write(result + "\n\n")
            time.sleep(1)

        except Exception as e:
            print(f"❌ Erreur batch {i+1} : {e}")

with open(MESSAGES_LOG, "w", encoding="utf-8") as f:
    json.dump(history, f, ensure_ascii=False, indent=2)

# 🔬 Test: envoyer une phrase sans prompt
new_sentence = "rue molière || Dénomination || Nouveau nom donné après 1802"
user_test_msg = {"role": "user", "content": new_sentence}
messages.append(user_test_msg)

print("\n🧠 TEST DE MÉMOIRE sans rappel de prompt :")
for m in messages[-2:]:
    print(f"[{m['role'].upper()}] {m['content']}\n")

try:
    memory_response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages
    )
    memory_result = memory_response["choices"][0]["message"]["content"]
    print("\n📥 RÉPONSE DU TEST :")
    print(memory_result)
except Exception as e:
    print(f"❌ Erreur pendant le test : {e}")

print("\n✅ Finie avec gestion de mémoire.")



📤 Batch 1 envoyé avec contexte :
[SYSTEM] Tu es un expert en traitement des documents historiques de Paris. Tu dois extraire les événements dans ce format :
'rue de Rivoli || Classement || Classement en 1801'
'rue de Rivoli || Dénomination || Nouveau nom en 1802'
N'oublie pas ce format pour chaque réponse.

[USER] Voici les événements à extraire :
avenue du maine || Historique || Cette voie est indiquée sur le plan de Vaugondy (1760)
impasse martini || Historique || Elle existait en 1652



KeyboardInterrupt: 

In [1]:
# Functions

def load_train_examples(filepath, max_total=60):
    examples = []
    with open(filepath, "r", encoding="utf-8") as f:
        for ligne in f:
            ligne = ligne.strip()
            if not ligne or ligne == ',':
                continue
            if ligne.endswith(','):
                ligne = ligne[:-1]
            try:
                obj = json.loads(ligne)
                examples.append(obj)
            except Exception as e:
                print("跳过异常行:", e, ligne)
            if len(examples) >= max_total:
                break
    return examples

def examples_to_prompt_str(exemples):
    return "\n\n".join(json.dumps(ex, ensure_ascii=False) for ex in exemples)

def parse_llm_output(raw_output):
    import json
    import re
    # 1. 提取所有```json ... ```块
    json_blocks = re.findall(r'```json\s*(\{.*?\})\s*```', raw_output, re.DOTALL)
    results = []
    if json_blocks:
        for block in json_blocks:
            try:
                obj = json.loads(block)
                results.append(obj)
            except Exception as e:
                print("块解析失败:", e, block)
        return results

    # 2. 提取所有裸JSON对象
    json_objs = re.findall(r'(\{(?:[^{}]|(?:\{[^{}]*\}))*\})', raw_output, re.DOTALL)
    if json_objs:
        for obj_str in json_objs:
            try:
                obj = json.loads(obj_str)
                results.append(obj)
            except Exception as e:
                print("裸对象解析失败:", e, obj_str)
        if results:
            return results

    # 3. 每行一个JSON对象
    lines = [l for l in raw_output.strip().splitlines() if l.strip()]
    for l in lines:
        print("尝试解析行:", l)
        try:
            obj = json.loads(l)
            results.append(obj)
        except Exception as e:
            print("行解析失败:", e, l)
    if results:
        return results

    # 4. 整体解析为JSON对象或数组
    try:
        arr = json.loads(raw_output)
        if isinstance(arr, dict):
            return [arr]
        if isinstance(arr, list):
            return arr
    except Exception as e:
        print("整体解析失败:", e, raw_output)
    return []

def try_batch(client, nom_modele, batch, batch_prompt, batch_size, chemin_sortie, i):
    print("本批次输入ID：", [rec["id"] for rec in batch])
    try:
        response = client.models.generate_content(
            model=nom_modele,
            contents=batch_prompt,
        )
        raw_output = response.text
        print(f"\n--- LLM Raw Output for Batch {i} (size={batch_size}) ---\n{raw_output}\n--- END ---\n")
        results = []
        try:
            results = json.loads(raw_output)
            if isinstance(results, dict):
                results = [results]
        except Exception:
            results = parse_llm_output(raw_output)
        # 过滤掉None和无id的
        results = [parsed for parsed in results if parsed and parsed.get("id")]
        print("解析后输出ID：", [parsed.get("id") for parsed in results])
        if not results or len(results) < len(batch):
            print(f"⚠️ 批量输出解析失败或条数不足，batch_size={batch_size}，应有{len(batch)}条，实际{len(results)}条，原始内容：\n", raw_output)
            return False
        else:
            with open(chemin_sortie, "a", encoding="utf-8") as fout:
                for parsed in results:
                    print(f"写入: {parsed.get('id', '[NO_ID]')}")
                    fout.write(json.dumps(parsed, ensure_ascii=False) + "\n")
            print(f"✅ Batch {i} ~ {i+len(batch)-1} 已写入 {len(results)} 条 (batch_size={batch_size})")
            return True
    except Exception as e:
        print(f"❌ Erreur lors du traitement du batch {i}-{i+batch_size}: {e}")
        return False

In [ ]:
import os
import json
import time
from google import genai
import openai

# ==================== CONFIGURATION ====================
work_dir = "C:/Users/jguo/Desktop/text2kgbench/text2kgbench"
os.chdir(work_dir) # ou le répertoire de travail approprié

openai.api_key = "xxxxxx" # Remplacez par votre clé API OpenAI

# Chemins des fichiers
dossier_train  = os.path.normpath(os.path.join(work_dir, "event_json_to_ttl/data/simplified_ground_truth.jsonl"))
dossier_test   = os.path.normpath(os.path.join(work_dir, "output.jsonl"))
dossier_sortie = os.path.normpath(os.path.join(work_dir, "llm_responses"))
prompt_file    = os.path.normpath(os.path.join(work_dir, "event_json_to_ttl/prompts/promptSimple.txt"))

# Paramètres du modèle et tokens
nom_modele   = "gpt-3.5-turbo"
MAX_TOKENS   = 6000
BATCH_SIZES  = [50, 25, 10, 5, 1]

os.makedirs(dossier_sortie, exist_ok=True)

with open(prompt_file, 'r', encoding='utf-8') as pf:
    template_prompt = pf.read().strip()

few_shot_examples = load_train_examples(dossier_train)
few_shot_text     = examples_to_prompt_str(few_shot_examples)

initial_system_full = {
    "role": "system",
    "content": template_prompt + "\n\nExemples :\n" + few_shot_text
}

# --- Prompt minimal à inclure à chaque requête ---
prompt_minimal = {
    "role": "system",
    "content": ("""
        "Vous êtes un assistant qui extrait des triplets RDF en respectant les exemples de few-shot. "
        "Pour chaque ligne JSON (JSONL) contenant "id" et "sent" (phrase d’événement), génère un unique objet JSONL :
     {
            "id": "<id_exact_de_ligne_d’entrée>",   ← NE CHANGE PAS CETTE VALEUR
            "response": "<phrase_originale>",
            "triples": [
                [<sujet>, <relation>, <objet>],
                …
            ]
            }

            **1. Relations existantes**
            isLandmarkType
            appearsOn
            disappearsOn
            hasAppearedRelationOn
            hasDisappearedRelationOn
            isClassifiedOn
            isLandmarkType
            isNumberedOn
            hasNew<AttributeType> (Name, Geometry)
            hasOld<AttributeType>
            has<AttributeType>ChangeOn
            hasTime
"""
    )
}

# Chargement des données de test
with open(dossier_test, 'r', encoding='utf-8') as fin:
    records = [json.loads(l) for l in fin if l.strip()]

# Estimation simple de tokens
def estimate_tokens(msgs):
    return sum(len(m['content'].split()) for m in msgs)

# Sortie
os.makedirs(dossier_sortie, exist_ok=True)
i = 0
file_idx = 1
base_name = os.path.splitext(os.path.basename(dossier_test))[0]
chemin_sortie = os.path.join(dossier_sortie, f"{base_name}_part{file_idx}.jsonl")

with open(chemin_sortie, 'w', encoding='utf-8') as fout:
    used_fewshot = False
    while i < len(records):
        if i > 0 and i % 10000 == 0:
            file_idx += 1
            fout.close()
            chemin_sortie = os.path.join(dossier_sortie, f"{base_name}_part{file_idx}.jsonl")
            fout = open(chemin_sortie, 'w', encoding='utf-8')

        # Préparer le contexte
        messages = []
        if not used_fewshot:
            messages.append(initial_system_full)
            # Afficher le prompt complet et les few-shot examples
            print("\n=== PROMPT COMPLET + EXEMPLES FEW-SHOT ENVOYÉS ===")
            for line in initial_system_full['content'].split('\n'):
                print(line)
        # Toujours ajouter le prompt minimal
        messages.append(prompt_minimal)


        # Ajouter l'historique (user+assistant) si disponible
        # (Aucun pour la première requête)
        # messages += historique, géré ci-dessous

        for batch_size in BATCH_SIZES:
            batch = records[i:i + batch_size]
            if not batch:
                continue

            user_content = "".join([f'Phrase (id={r["id"]}): {r["sent"]}\n' for r in batch])
            user_msg = {"role": "user", "content": user_content}
            messages.append(user_msg)

            # Si trop long, garder seulement prompt_minimal + user_msg
            if estimate_tokens(messages) > MAX_TOKENS:
                messages = [prompt_minimal, user_msg]

            # Afficher tout le contenu envoyé
            print("\n=== MESSAGES ENVOYÉS AU MODÈLE ===")
            for msg in messages:
                print(f"[{msg['role']}] {msg['content']}\n")

            # Appel API
            try:
                response = openai.ChatCompletion.create(
                    model=nom_modele,
                    messages=messages
                )
                result = response['choices'][0]['message']['content']

                # Afficher la réponse brute
                print("\n=== RÉPONSE DU MODÈLE ===")
                print(result)

                # Sauvegarder
                fout.write(json.dumps({"id": batch[0]["id"], "response": result}, ensure_ascii=False) + "\n")

                # Conserver historique
                messages.append({"role": "assistant", "content": result})
                used_fewshot = True
                i += len(batch)
                time.sleep(1)
                break
            except Exception as e:
                print(f"Erreur batch {i} taille {batch_size} : {e}")
                time.sleep(1)
        else:
            print(f"Tous les batch_sizes ont échoué pour l’index {i}, on saute.")
            i += 1

print("\nTraitement terminé.")



=== PROMPT COMPLET + EXEMPLES FEW-SHOT ENVOYÉS ===
Tu es un assistant qui extrait des triplets RDF.  
Pour chaque ligne JSON (JSONL) contenant "id" et "sent" (phrase d’événement), génère un unique objet JSONL :

{
  "id": "<id_exact_de_ligne_d’entrée>",   ← NE CHANGE PAS CETTE VALEUR
  "response": "<phrase_originale>",
  "triples": [
    [<sujet>, <relation>, <objet>],
    …
  ]
}

**1. Relations existantes**
isLandmarkType
appearsOn
disappearsOn
hasAppearedRelationOn
hasDisappearedRelationOn
isClassifiedOn
isLandmarkType
isNumberedOn
hasNew<AttributeType> (Name, Geometry)
hasOld<AttributeType>
has<AttributeType>ChangeOn
hasTime

**Règles essentielles :**
- Copie la phrase d’entrée dans `"response"`.
- N’ajouter de `hasTime` que pour une date ISO complète, et en un seul exemplaire. Si aucune date n'existe, utilise "<sujet> hasTime noTime"


**5. Contrainte anti-hallucination :**
- N’invente pas de relations ou concepts hors liste.  
- Tout triple ne respectant pas ces relations doit ê

In [20]:
work_dir = "C:/Users/jguo/Desktop/text2kgbench/text2kgbench"
os.chdir(work_dir)

In [ ]:
# test la mémoire de modèle sans prompt
# Ce script utilise VetexAI pour extraire des événements historiques de Paris 
from google import genai
import time
import json
dossier_train   = os.path.normpath(os.path.join(work_dir, "ground_truth/ground_truth.jsonl"))
dossier_test = os.path.normpath(os.path.join(work_dir, "output.jsonl"))
dossier_sortie  = os.path.normpath(os.path.join(work_dir, "llm_responses"))

prompt_file = os.path.normpath(os.path.join(work_dir, "event_json_to_ttl/prompts/promptSimple.txt"))
BATCH_SIZE = 50
OUTPUT_FILE = "extraction_output.txt"
MESSAGES_LOG = "chat_history.json"
MAX_TOKENS_THRESHOLD = 6000
nom_modele = "gemini-2.5-pro-preview-05-06"

client = genai.Client(
    vertexai=True,
    project="ignf-simv-inference",
    location="us-central1"
)

system_prompt = {
    "role": "system",
    "content": prompt_file
}

# 📓 Event list
evenements = []
with open("dossier_test.txt", "r", encoding="utf-8") as f:
    for line in f:
        evenement = line.strip()
        if evenement:  
            evenements.append(evenement)

NB_BATCHES = len(evenements) // BATCH_SIZE + int(len(evenements) % BATCH_SIZE != 0)

# Token estimate

def estimate_tokens(msg_list):
    return sum(len(msg["content"].split()) for msg in msg_list)

messages = []
history = []

with open(OUTPUT_FILE, "w", encoding="utf-8") as f_out:
    for i in range(NB_BATCHES):
        start = i * BATCH_SIZE
        end = min(start + BATCH_SIZE, len(evenements))
        batch_text = "\n".join(evenements[start:end])
        user_msg = {"role": "user", "content": f"Voici les événements à extraire :\n{batch_text}"}

        if i == 0:
            messages.append(system_prompt)

        messages.append(user_msg)

        if estimate_tokens(messages) > MAX_TOKENS_THRESHOLD:
            print("\n⚠️ Token limite atteinte. Reboot avec prompt.")
            messages = [system_prompt, user_msg]

        print(f"\n📤 Batch {i+1} envoyé avec contexte :")
        for m in messages[-2:]:
            print(f"[{m['role'].upper()}] {m['content']}\n")

        try:
            response = client.models.generate_content(
            model=nom_modele,
            contents=evenements,)

            
            result = response["choices"][0]["message"]["content"]

            print(f"📥 Réponse batch {i+1} :\n{result}\n")

            messages.append({"role": "assistant", "content": result})
            history.append({"user": user_msg["content"], "assistant": result})

            f_out.write(f"--- Résultats {start}-{end} ---\n")
            f_out.write(result + "\n\n")
            time.sleep(1)

        except Exception as e:
            print(f"❌ Erreur batch {i+1} : {e}")

with open(MESSAGES_LOG, "w", encoding="utf-8") as f:
    json.dump(history, f, ensure_ascii=False, indent=2)

# 🔬 Test: envoyer une phrase sans prompt
new_sentence = "rue molière || Dénomination || Nouveau nom donné après 1802"
user_test_msg = {"role": "user", "content": new_sentence}
messages.append(user_test_msg)

print("\n🧠 TEST DE MÉMOIRE sans rappel de prompt :")
for m in messages[-2:]:
    print(f"[{m['role'].upper()}] {m['content']}\n")

try:
    memory_response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages
    )
    memory_result = memory_response["choices"][0]["message"]["content"]
    print("\n📥 RÉPONSE DU TEST :")
    print(memory_result)
except Exception as e:
    print(f"❌ Erreur pendant le test : {e}")

print("\n✅ Finie avec gestion de mémoire.")



📤 Batch 1 envoyé avec contexte :
[SYSTEM] 
            Tu es un assistant qui extrait des triplets RDF en respectant l’ontologie PeGazUs.  
            Pour chaque ligne JSON (JSONL) contenant "id" et "sent" (phrase d’événement), génère un unique objet JSONL :

            {
            "id": "<id_exact_de_ligne_d’entrée>",   ← NE CHANGE PAS CETTE VALEUR
            "response": "<phrase_originale>",
            "triples": [
                [<sujet>, <relation>, <objet>],
                …
            ]
            }

            **1. Relations existantes**
            isLandmarkType
            appearsOn
            disappearsOn
            hasAppearedRelationOn
            hasDisappearedRelationOn
            isClassifiedOn
            isLandmarkType
            isNumberedOn
            hasNew<AttributeType> (Name, Geometry)
            hasOld<AttributeType> (Name, Geometry)
            has<AttributeType>ChangeOn
            hasTime

            **Règles essentielles :**
            

KeyboardInterrupt: 

In [8]:
work_dir = "C:/Users/jguo/Desktop/text2kgbench/text2kgbench"
os.chdir(work_dir)

In [6]:
nom_modele = "gemini-2.5-pro-preview-05-06"

client = genai.Client(
    vertexai=True,
    project="ignf-simv-inference",
    location="us-central1"
)

response = client.models.generate_content(
    model=nom_modele,
    contents="Bonjour ! Raconte-moi une blague",
)

print(response.text)

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029E63DD6990>, 'Connection to oauth2.googleapis.com timed out. (connect timeout=120)'))

In [7]:
# Functions

def load_train_examples(filepath, max_total=60):
    examples = []
    with open(filepath, "r", encoding="utf-8") as f:
        for ligne in f:
            ligne = ligne.strip()
            if not ligne or ligne == ',':
                continue
            if ligne.endswith(','):
                ligne = ligne[:-1]
            try:
                obj = json.loads(ligne)
                examples.append(obj)
            except Exception as e:
                print("跳过异常行:", e, ligne)
            if len(examples) >= max_total:
                break
    return examples

def examples_to_prompt_str(exemples):
    return "\n\n".join(json.dumps(ex, ensure_ascii=False) for ex in exemples)

def parse_llm_output(raw_output):
    import json
    import re
    # 1. 提取所有```json ... ```块
    json_blocks = re.findall(r'```json\s*(\{.*?\})\s*```', raw_output, re.DOTALL)
    results = []
    if json_blocks:
        for block in json_blocks:
            try:
                obj = json.loads(block)
                results.append(obj)
            except Exception as e:
                print("块解析失败:", e, block)
        return results

    # 2. 提取所有裸JSON对象
    json_objs = re.findall(r'(\{(?:[^{}]|(?:\{[^{}]*\}))*\})', raw_output, re.DOTALL)
    if json_objs:
        for obj_str in json_objs:
            try:
                obj = json.loads(obj_str)
                results.append(obj)
            except Exception as e:
                print("裸对象解析失败:", e, obj_str)
        if results:
            return results

    # 3. 每行一个JSON对象
    lines = [l for l in raw_output.strip().splitlines() if l.strip()]
    for l in lines:
        print("尝试解析行:", l)
        try:
            obj = json.loads(l)
            results.append(obj)
        except Exception as e:
            print("行解析失败:", e, l)
    if results:
        return results

    # 4. 整体解析为JSON对象或数组
    try:
        arr = json.loads(raw_output)
        if isinstance(arr, dict):
            return [arr]
        if isinstance(arr, list):
            return arr
    except Exception as e:
        print("整体解析失败:", e, raw_output)
    return []

def try_batch(client, nom_modele, batch, batch_prompt, batch_size, chemin_sortie, i):
    print("本批次输入ID：", [rec["id"] for rec in batch])
    try:
        response = client.models.generate_content(
            model=nom_modele,
            contents=batch_prompt,
        )
        raw_output = response.text
        print(f"\n--- LLM Raw Output for Batch {i} (size={batch_size}) ---\n{raw_output}\n--- END ---\n")
        results = []
        try:
            results = json.loads(raw_output)
            if isinstance(results, dict):
                results = [results]
        except Exception:
            results = parse_llm_output(raw_output)
        # 过滤掉None和无id的
        results = [parsed for parsed in results if parsed and parsed.get("id")]
        print("解析后输出ID：", [parsed.get("id") for parsed in results])
        if not results or len(results) < len(batch):
            print(f"⚠️ 批量输出解析失败或条数不足，batch_size={batch_size}，应有{len(batch)}条，实际{len(results)}条，原始内容：\n", raw_output)
            return False
        else:
            with open(chemin_sortie, "a", encoding="utf-8") as fout:
                for parsed in results:
                    print(f"写入: {parsed.get('id', '[NO_ID]')}")
                    fout.write(json.dumps(parsed, ensure_ascii=False) + "\n")
            print(f"✅ Batch {i} ~ {i+len(batch)-1} 已写入 {len(results)} 条 (batch_size={batch_size})")
            return True
    except Exception as e:
        print(f"❌ Erreur lors du traitement du batch {i}-{i+batch_size}: {e}")
        return False

In [1]:
# ==================== CONFIGURATION ====================
work_dir = "C:/Users/jguo/Desktop/text2kgbench/text2kgbench"
os.chdir(work_dir)
dossier_train   = os.path.normpath(os.path.join(work_dir, "event_json_to_ttl/data/simplified_ground_truth.jsonl"))
dossier_test = os.path.normpath(os.path.join(work_dir, "output.jsonl"))
dossier_sortie  = os.path.normpath(os.path.join(work_dir, "llm_responses"))

prompt_file = os.path.normpath(os.path.join(work_dir, "event_json_to_ttl/prompts/promptSimple.txt"))
nom_modele = "gemini-2.5-pro-preview-05-06"

client = genai.Client(
    vertexai=True,
    project="ignf-simv-inference",
    location="us-central1"
)

os.makedirs(dossier_sortie, exist_ok=True)

with open(prompt_file, "r", encoding="utf-8") as prompt:
    template_prompt = prompt.read()

# ========== TRAITEMENT PRINCIPAL ==========

few_shot_examples = load_train_examples(dossier_train)
few_shot_text     = examples_to_prompt_str(few_shot_examples)

with open(dossier_test, "r", encoding="utf-8") as fin:
    records = [json.loads(ligne) for ligne in fin if ligne.strip()]

base = os.path.splitext(os.path.basename(dossier_test))[0]
i = 0
file_idx = 1
chemin_sortie = os.path.normpath(os.path.join(
    dossier_sortie, f"{base}_llm_responses_part{file_idx}.jsonl"
))

while i < len(records):
    # 每500条切换一个输出文件
    if i % 10000 == 0 and i > 0:
        print(f"---- 切换到下一个输出文件 ----")
        file_idx += 1
        chemin_sortie = os.path.normpath(os.path.join(
            dossier_sortie, f"{base}_llm_responses_part{file_idx}.jsonl"
        ))

    # for batch_size in [20,5,1]:
    for batch_size in [1]:
        batch = records[i:i+batch_size]
        if not batch:
            break
        batch_prompt = template_prompt + "\n\nExemples :\n" + few_shot_text + "\n\n"
        for rec in batch:
            batch_prompt += f'Phrase (id={rec["id"]}): {rec["sent"]}\n'
        batch_prompt += "\nGénère pour chaque phrase un objet JSON sur une ligne, au format JSONL (une ligne par objet JSON, pas de liste, pas de crochets).\n"
        token_count = len(batch_prompt) // 4
        print(f"Batch {i} ~ {i+len(batch)-1} (size={batch_size}) 估算token数: {token_count}")

        print(f"\n=== Batch {i} ~ {i+len(batch)-1} Prompt (size={batch_size}) ===\n{batch_prompt}\n")
       
        ok = try_batch(client, nom_modele, batch, batch_prompt, batch_size, chemin_sortie, i)
        if ok:
            i += batch_size
            break
        else:
            print(f"⚠️ Batch size {batch_size} 失败，尝试更小的 batch...")
            time.sleep(2)
    else:
        # 如果1也失败，则无限重试
        print(f"❌ 连 batch_size=1 都失败，开始无限重试第{i}条数据")
        while True:
            batch = records[i:i+1]
            batch_prompt = template_prompt + "\n\nExemples :\n" + few_shot_text + "\n\n"
            batch_prompt += f'Phrase (id={batch[0]["id"]}): {batch[0]["sent"]}\n'
            batch_prompt += "\nGénère pour chaque phrase un objet JSON sur une ligne, au format JSONL (une ligne par objet JSON, pas de liste, pas de crochets).\n"
            token_count = len(batch_prompt) // 4
            print(f"重试 Batch {i} (size=1) 估算token数: {token_count}")
            ok = try_batch(client, nom_modele, batch, batch_prompt, 1, chemin_sortie, i)
            if ok:
                i += 1
                break
            else:
                print(f"⚠️ 单条重试失败，3秒后再试...")
                time.sleep(3)

    time.sleep(5)

NameError: name 'os' is not defined

In [ ]:
# 假设
output_ids = ['11815_dénomination', '11815_ouverture', '12621_historique', '9529_historique', '9529_classement', '14371_historique', '9673_historique', '9673_classement', '9673_ouverture', '9673_numérotation', '11703_classement', '11703_ouverture', '9651_dénomination', '9651_ouverture', '14312_historique', '14414_historique', '14494_historique', '14566_historique', '14574_historique', '13914_classement', '12250_historique', '12250_ouverture', '15466_dénomination', '12925_historique', '12925_ouverture', '14037_historique', '14037_classement', '13936_classement', '10518_ouverture', '14561_historique', '11029_historique', '11029_dénomination', '11029_ouverture', '13957_historique', '13957_classement', '12669_historique', '12669_classement', '11306_historique', '11306_ouverture', '12573_historique_1', '12573_historique_2', '12573_dénomination', '12573_numérotation', '11972_historique', '11972_ouverture', '9240_historique', '9240_dénomination', '9240_ouverture', '13902_classement', '13902_ouverture']
input_ids = ['11815_dénomination', '11815_ouverture', '12621_historique', '9529_historique', '9529_classement', '14371_historique', '9673_historique', '9673_classement', '9673_ouverture', '9673_numérotation', '11703_classement', '11703_ouverture', '9651_dénomination', '9651_ouverture', '14312_historique', '14414_historique', '14494_historique', '14566_historique', '14574_historique', '13914_classement', '12250_historique', '12250_ouverture', '15466_dénomination', '12925_historique', '12925_ouverture', '14037_historique', '14037_classement', '13936_classement', '10518_ouverture', '14561_historique', '11029_historique', '11029_dénomination', '11029_ouverture', '13957_historique', '13957_classement', '12669_historique', '12669_classement', '11306_historique', '11306_ouverture', '12573_historique_1', '12573_historique_2', '12573_dénomination', '12573_numérotation', '11972_historique', '11972_ouverture', '9240_historique', '9240_dénomination', '9240_ouverture', '13902_classement', '13902_ouverture']

missing_ids = [id for id in input_ids if id not in output_ids]
print("被跳过/遗漏的ID：", missing_ids)
print("共遗漏：", len(missing_ids), "条")

# Generating LLM responses

In [ ]:
response = get_llm_response(prompt_example)

In [ ]:
print(response)

# Parsing responses

In [ ]:
# go through responses of LLM responses
# parse_llm_response(response)
#

# Evaluating the triples with the ground truth

In [ ]:
ground_truth_files = [
    "onto_1_movie": "data/ground_truth/ont_1_movie_ground_truth.jsonl",
    "onto_2_music": "data/ground_truth/ont_1_music_ground_truth.jsonl"
}

In [ ]:


import json
with open("/Users/jguo/Desktop/text2kgbench/text2kgbench/text2kgbench/text2kgbench/llm_responses/ont_1_rue_mizon_test_llm_responses.jsonl",encoding="utf-8") as f:
    for i,line in enumerate(f):
        rec = json.loads(line)
        print("id=", rec["id"], "; triples=", rec.get("triples"))
        if i>=2: break


In [ ]:
rec.get("triples")


In [ ]:
import json
with open("/Users/jguo/Desktop/text2kgbench/text2kgbench/text2kgbench/text2kgbench/ground_truth/ont_1_rue_mizon_ground_truth.jsonl","r",encoding="utf-8") as f:
    for i,line in enumerate(f):
        rec = json.loads(line)
        print(rec["id"], rec.get("triples"))
        if i>=2: break
